<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) | [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) | [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.02-Data-Indexing-and-Selection.ipynb"><img align="left" src ="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# データの索引付けと選択

[Chapter 2](02.00-Introduction-to-NumPy.ipynb) では、NumPy 配列の値にアクセス、設定、および変更するためのメソッドとツールについて詳しく調べました。
これらには、インデックス作成 (例: arr[2, 1]``)、スライシング (例: arr[2, 1]``))、マスキング (例: arr[2, 1]``))、派手なインデックス作成 (例: arr[2, 1]``)、およびそれらの組み合わせ (例: arr[2, 1]``).
ここでは、Pandas ``Series`` および ``DataFrame`` オブジェクトの値にアクセスして変更する同様の方法を見ていきます。
NumPy パターンを使用したことがある場合、Pandas の対応するパターンは非常に親しみやすいものですが、注意すべきいくつかの癖があります。

1 次元の ``Series`` オブジェクトの単純なケースから始めて、より複雑な 2 次元の ``DataFrame`` オブジェクトに移ります。

## シリーズのデータ​​選択

前のセクションで見たように、Series オブジェクトは多くの点で 1 次元の NumPy 配列のように機能し、多くの点で標準の Python 辞書のように機能します。
これら 2 つの重複するアナロジーを念頭に置くと、これらの配列でのデータのインデックス付けと選択のパターンを理解するのに役立ちます。

### 辞書としてのシリーズ

ディクショナリのように、Series オブジェクトはキーのコレクションから値のコレクションへのマッピングを提供します。

In [1]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [2]:
data['b']

0.5

辞書のような Python 式とメソッドを使用して、キー/インデックスと値を調べることもできます。

In [3]:
'a' in data

True

In [4]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Series オブジェクトは、辞書のような構文で変更することもできます。
新しいキーを代入することで辞書を拡張できるように、新しいインデックス値を代入することで ``Series`` を拡張できます:

In [6]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

オブジェクトのこの簡単な可変性は便利な機能です。Pandas は内部で、実行する必要がある可能性のあるメモリ レイアウトとデータ コピーに関する決定を行っています。通常、ユーザーはこれらの問題について心配する必要はありません。

### 1 次元配列としてのシリーズ

``Series`` は、この辞書のようなインターフェイスに基づいて構築されており、NumPy 配列と同じ基本的なメカニズム (つまり、*スライス*、*マスキング*、*ファンシー インデックス*) を介して配列スタイルのアイテム選択を提供します。
これらの例は次のとおりです。

In [7]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [8]:
# slicing by implicit integer index
data[0:2]

a    0.25
b    0.50
dtype: float64

In [9]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [10]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

これらの中で、スライスが最も混乱の原因となる可能性があります。
明示的なインデックス (つまり、data['a':'c']``)) でスライスする場合、最終的なインデックスはスライスに *含まれる* のに対し、暗黙的なインデックス (つまり、data['a':'c']``)) でスライスする場合、最終的なインデックスはスライスから *除外されることに注意してください。 .

### インデクサー: loc、iloc、および ix

これらのスライスとインデックス付けの規則は、混乱の元になる可能性があります。
たとえば、 Series に明示的な整数インデックスがある場合、 data[1] などのインデックス操作では明示的なインデックスが使用され、 data[1:3] などのスライス操作では明示的なインデックスが使用されます。暗黙的な Python スタイルのインデックスを使用します。

In [11]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [12]:
# explicit index when indexing
data[1]

'a'

In [13]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

整数インデックスの場合に混乱が生じる可能性があるため、Pandas は特定のインデックス スキームを明示的に公開するいくつかの特別な *indexer* 属性を提供します。
これらは関数メソッドではなく、特定のスライス インターフェイスを ``Series`` 内のデータに公開する属性です。

まず、 ``loc`` 属性により、常に明示的なインデックスを参照するインデックス作成とスライスが可能になります。

In [14]:
data.loc[1]

'a'

In [15]:
data.loc[1:3]

1    a
3    b
dtype: object

``iloc`` 属性を使用すると、暗黙的な Python スタイルのインデックスを常に参照するインデックス作成とスライスが可能になります。

In [16]:
data.iloc[1]

'b'

In [17]:
data.iloc[1:3]

3    b
5    c
dtype: object

3 番目のインデックス属性 ``ix`` は 2 つのハイブリッドであり、``Series`` オブジェクトの場合、標準の ``[]`` ベースのインデックス作成と同等です。
``ix`` インデクサーの目的は、後で説明する ``DataFrame`` オブジェクトのコンテキストでより明確になります。

Python コードの指針となる原則の 1 つは、「明示的は暗黙的よりも優れている」ということです。
loc と iloc の明示的な性質により、きれいで読みやすいコードを維持するのに非常に役立ちます。特に整数インデックスの場合、コードを読みやすく理解しやすくするためと、インデックス作成とスライスの規則が混在していることによる微妙なバグを防ぐために、これらの両方を使用することをお勧めします。

## DataFrame でのデータ選択

``DataFrame`` は多くの点で 2 次元または構造化配列のように機能し、他の方法では同じインデックスを共有する ``Series`` 構造の辞書のように機能することを思い出してください。
これらの類推は、この構造内でのデータ選択を検討する際に覚えておくと役立ちます。

### 辞書としての DataFrame

最初に検討する類推は、関連する ``Series`` オブジェクトの辞書としての ``DataFrame`` です。
州の地域と人口の例に戻りましょう。

In [18]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135
New York,141297,19651127
Texas,695662,26448193


``DataFrame`` の列を構成する個々の ``Series`` には、列名の辞書スタイルのインデックスを介してアクセスできます。

In [19]:
data['area']

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

同様に、文字列である列名で属性スタイルのアクセスを使用できます。

In [20]:
data.area

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

この属性スタイルの列アクセスは、辞書スタイルのアクセスとまったく同じオブジェクトに実際にアクセスします。

In [21]:
data.area is data['area']

True

これは便利な省略形ですが、すべての場合に機能するわけではないことに注意してください。
たとえば、列名が文字列でない場合、または列名が ``DataFrame`` のメソッドと競合する場合、この属性スタイルのアクセスは不可能です。
たとえば、 ``DataFrame`` には ``pop()`` メソッドがあるため、 ``data.pop`` は ``"pop"`` カラムではなく、これを指します:

In [22]:
data.pop is data['pop']

False

特に、属性を介して列の割り当てを試みる誘惑を避ける必要があります (つまり、data['pop'] = z`` rather than ``data.pop = z``) を使用します)。

前に説明した ``Series`` オブジェクトと同様に、この辞書スタイルの構文を使用してオブジェクトを変更することもできます。この場合、新しい列を追加します。

In [23]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740


これは ``Series`` オブジェクト間の要素ごとの算術演算の簡単な構文のプレビューを示しています。これについては、[Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) でさらに掘り下げます。

### 2 次元配列としての DataFrame

前に述べたように、拡張された 2 次元配列として ``DataFrame`` を表示することもできます。
``values`` 属性を使用して、生の基になるデータ配列を調べることができます。

In [24]:
data.values

array([[  4.23967000e+05,   3.83325210e+07,   9.04139261e+01],
       [  1.70312000e+05,   1.95528600e+07,   1.14806121e+02],
       [  1.49995000e+05,   1.28821350e+07,   8.58837628e+01],
       [  1.41297000e+05,   1.96511270e+07,   1.39076746e+02],
       [  6.95662000e+05,   2.64481930e+07,   3.80187404e+01]])

この図を念頭に置いて、おなじみの配列のような多くの観察を ``DataFrame`` 自体で行うことができます。
たとえば、完全な ``DataFrame`` を転置して、行と列を入れ替えることができます。

In [25]:
data.T

,California,Florida,Illinois,New York,Texas
area,4.239670e+05,1.703120e+05,1.499950e+05,1.412970e+05,6.956620e+05
pop,3.833252e+07,1.955286e+07,1.288214e+07,1.965113e+07,2.644819e+07
density,9.041393e+01,1.148061e+02,8.588376e+01,1.390767e+02,3.801874e+01


しかし、 ``DataFrame`` オブジェクトのインデックス付けに関して言えば、列の辞書スタイルのインデックス付けにより、単純に NumPy 配列として扱うことができないことは明らかです。
特に、単一のインデックスを配列に渡すと、行にアクセスします。

In [26]:
data.values[0]

array([  4.23967000e+05,   3.83325210e+07,   9.04139261e+01])

単一の「インデックス」を ``DataFrame`` に渡すと、列にアクセスします:

In [27]:
data['area']

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

したがって、配列スタイルのインデックス付けには、別の規則が必要です。
ここでも Pandas は、前述の loc 、 iloc 、および ix インデクサーを使用します。
``iloc`` インデクサを使用すると、基になる配列を単純な NumPy 配列であるかのように (暗黙的な Python スタイルのインデックスを使用して) インデックス付けできますが、 ``DataFrame`` インデックスと列ラベルは結果で維持されます。

In [28]:
data.iloc[:3, :2]

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


同様に、 ``loc`` インデクサーを使用すると、配列のようなスタイルで基になるデータにインデックスを付けることができますが、明示的なインデックスと列名を使用します。

In [29]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


ix インデクサーでは、次の 2 つのアプローチのハイブリッドが可能です。

In [30]:
data.ix[:3, :'pop']

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


整数インデックスの ``ix`` インデクサーは、整数インデックスの ``Series`` オブジェクトについて説明したのと同じ潜在的な混乱の原因になることに注意してください。

これらのインデクサーでは、使い慣れた NumPy スタイルのデータ アクセス パターンを使用できます。
たとえば、 ``loc`` インデクサでは、次のようにマスキングとファンシー インデックスを組み合わせることができます。

In [31]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
Florida,19552860,114.806121
New York,19651127,139.076746


これらのインデックス付け規則のいずれかを使用して、値を設定または変更することもできます。これは、NumPy での作業に慣れている標準的な方法で行われます。

In [32]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,38332521,90.000000
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740


Pandas データ操作の流暢さを高めるために、単純な ``DataFrame`` に時間を費やし、これらのさまざまなインデックス作成アプローチで許可されているインデックス作成、スライシング、マスキング、およびファンシー インデックス作成の種類を調べることをお勧めします。

### 追加の索引付け規則

前述の説明と矛盾しているように見える追加のインデックス付け規則がいくつかありますが、実際には非常に役立ちます。
まず、*indexing* は列を参照しますが、*slicing* は行を参照します。

In [33]:
data['Florida':'Illinois']

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


このようなスライスは、インデックスではなく番号で行を参照することもできます。

In [34]:
data[1:3]

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


同様に、直接マスキング操作も列単位ではなく行単位で解釈されます。

In [35]:
data[data.density > 100]

,area,pop,density
Florida,170312,19552860,114.806121
New York,141297,19651127,139.076746


これら 2 つの規則は、NumPy 配列の規則と構文的に類似しており、Pandas 規則の型には正確に適合しない場合がありますが、それでも実際には非常に役立ちます。

<!--ナビゲーション-->
< [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) | [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) | [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.02-Data-Indexing-and-Selection.ipynb"><img align="left" src ="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
